# Oemof-thermal issue 
## The StratifiedThermalStorage class

Requirements:
- oemof.network==0.5.0
- oemof.solph==0.5.5
- oemof.thermal==0.0.6

Problem Description: 
Trying to recreate the example from the oemof-thermal for the **StratifiedThermalStorage** class, the code did not run as expected. There were error messages, that all involved the 
class attributes of the **StratifiedThermalStorage** class, that are not defined in the **Node** class.
Example of the error message:


        File c:\Users\To\AppData\Local\Programs\Python\Python312\Lib\site-packages\oemof\thermal\facades.py:79, in Facade.__init__(self, *args, **kwargs)
     75 self.type = kwargs.get("type")
     77 required = kwargs.pop("_facade_requires_", [])
          ---> 79 super().__init__(*args, **kwargs)
     81 self.subnodes = []
     82 EnergySystem.signals[EnergySystem.add].connect(
     83     add_subnodes, sender=self
     84 )

          TypeError: Node.__init__() got an unexpected keyword argument 'bus'

I suppose the error arises because the **StratifiedThermalStorage** class calls the **Facade** class. The **Facade** class inherits from the **Node** class and tries
to hand over the kwargs and args to the **Node** class in line 79 of the facade.py file in oemof-thermal:

          super().__init__(*args, **kwargs)

However the Node class does NOT accept kwargs or args. And the kwargs for **StratifiedThermalStorage** (like 'bus' or 'diameter') do not exist in the **Node** class. 
This results in the error message.

Possibly, there existed an oemof-version once, where **kwargs and *args could be passed to the Node class.

The code below should recreate the error message


In [1]:

import oemof.solph as solph
from oemof.thermal.facades import StratifiedThermalStorage
import pandas as pd


ES = solph.EnergySystem(timeindex=pd.date_range('1/1/2024', periods=10, freq='h'),infer_last_interval=True)



#bus,source,sink
bus_heat = solph.buses.Bus(label='bus_heat')
heatsource= solph.components.Source(label='heatsource', outputs={bus_heat: solph.flows.Flow()})
sinkHeat= solph.components.Sink(label='sinkHeat', inputs={bus_heat: solph.flows.Flow()})

#StratifiedThermalStorage
thermal_storage = StratifiedThermalStorage(
    label='thermal_storage',
    bus=bus_heat,
    diameter=3,
    height=5,
    temp_h=95,
    temp_c=60,
    temp_env=10,
    u_value=3,
    min_storage_level=0.05,
    max_storage_level=0.95,
    capacity=1,
    efficiency=0.9,
    marginal_cost=0.0001,
    initial_storage_level=0.05,
   
)
#adding the components to the energy system
ES.add(bus_heat,heatsource,sinkHeat,thermal_storage)

#creating the optimization model
model = solph.Model(ES)
output = model.solve(solver = 'cbc', solve_kwargs={'tee': False})
results = solph.processing.results(model)





TypeError: Node.__init__() got an unexpected keyword argument 'bus'

# suggested Solution

When altering the line 79 in the facade.py file (in the oemof.thermal package, that is responsible for the error) to only pass the 'label' argument to the __Node__ class, the code runs without errors.

        #super().__init__(*args, **kwargs)
        #now insert:
        node_kwargs={'label':kwargs.get('label')}
        super().__init__(**node_kwargs)

I would be interested if you have different approaches on how to resolve this issue. Also, i assumed that only the label is a necessary kwarg (since input and ouput are later defined in the StratifiedThermalStorage class), but i wonder if there are cases where that is not sufficient. 



